# Peripherals

In [1]:
%register serial:///dev/ttyAMA1
%connect robot-stm32

Connected to robot-stm32 @ serial:///dev/ttyAMA1


## GPIO

List of available GPIOs and their capabilities:

In [1]:
# pin alternate functions

from pyb import Pin

print("{:15} {:4} {}".format('board name', 'name', 'alternate functions'))
for p in sorted(dir(Pin.board)):
    if '__' in p: continue
    pin = Pin(p)
    names = pin.names()
    print("{:15} {:4} {}".format(names[1], names[0], pin.af_list()))

board name      name alternate functions
AIN1            C5   []
AIN2            C1   []
AUX             B13  [Pin.AF1_TIM1, Pin.AF5_SPI2, Pin.AF7_USART3]
BATTERY_MONITOR A3   [Pin.AF1_TIM2, Pin.AF2_TIM5, Pin.AF3_TIM9, Pin.AF7_USART2]
BIN1            C3   [Pin.AF5_SPI2]
BIN2            A4   [Pin.AF5_SPI1, Pin.AF7_USART2]
BOOT1           B2   []
D8              C0   []
ENC_A1          A6   [Pin.AF1_TIM1, Pin.AF2_TIM3, Pin.AF3_TIM8, Pin.AF5_SPI1, Pin.AF9_TIM13]
ENC_A2          A7   [Pin.AF1_TIM1, Pin.AF2_TIM3, Pin.AF3_TIM8, Pin.AF5_SPI1, Pin.AF9_TIM14]
ENC_B1          B6   [Pin.AF2_TIM4, Pin.AF4_I2C1]
ENC_B2          B7   [Pin.AF2_TIM4, Pin.AF4_I2C1]
FLASH_CS        A15  [Pin.AF1_TIM2, Pin.AF1_TIM2, Pin.AF5_SPI1]
FLASH_MISO      B4   [Pin.AF2_TIM3, Pin.AF5_SPI1]
FLASH_MOSI      B5   [Pin.AF2_TIM3, Pin.AF5_SPI1]
FLASH_SCK       B3   [Pin.AF1_TIM2, Pin.AF5_SPI1]
NC_A0           A0   [Pin.AF1_TIM2, Pin.AF1_TIM2, Pin.AF2_TIM5, Pin.AF3_TIM8, Pin.AF7_USART2]
NC_A1           A1   [Pin.AF1_TIM2,

## Motor Controller

The motor controller comprises two H-bridges to control the speed and direction of the motors and magnetic encoders to capture the rotational speed.

### H-bridges

The figure below shows the TB6612 motor controller. It is wired to the STM32 as follows:

* Channel A control: PWMA, AIN1, AIN2
* Channel B control: PWMB, BIN1, BIN2

```{figure} figures/TB6612-breakout.jpg
:width: 300px
:name: h-bridge

[TB6612](https://www.pololu.com/product/713) dual H-bridge motor controller breakout.
```

The `xIN1` and `xIN2` determine its state as follows:

| Function | IN1 | IN2 | Notes |
|----------|-----|-----|-------|
| Forward  |  L  |  H  |  
| Reverse  |  H  |  L  |
| Brake    |  H  |  H  | speed=0 has same effect
| Coast (Z)|  L  |  L  |

The TB6612 class maps the desired speed input to the appropriate control signals:

In [1]:
!cat code/stm32/lib/tb6612.py

"""
TB6612 motor driver.

Example:

from pyb import Pin, Timer
from tb6612 import TB6612
import time

freq = 10_0000
tim = Timer(8, freq=freq)

motor1 = TB6612(
    tim.channel(3, Timer.PWM_INVERTED, pin=Pin('PWM_A')),
    Pin('AIN1', mode=Pin.OUT_PP),
    Pin('AIN2', mode=Pin.OUT_PP)
)

motor2 = TB6612(
    tim.channel(1, Timer.PWM_INVERTED, pin=Pin('PWM_B')),
    Pin('BIN1', mode=Pin.OUT_PP),
    Pin('BIN2', mode=Pin.OUT_PP)
)

nstby = Pin('NSTBY', mode=Pin.OUT_PP)
nstby.value(1)

motor1.speed(30)
motor2.speed(-83)

time.sleep(5)
nstby.value(0)
"""

class TB6612:

    def __init__(self, pwm, in1, in2):
        self.pwm = pwm
        self.in1 = in1
        self.in2 = in2

    def speed(self, speed:float):
        """Set motor speed (duty cycle), range +/- 100."""
        if speed < 0:
            self.in1.value(0)
            self.in2.value(1)
        else:
            self.in1.value(1)
            self.in2.value(0)
        speed = min(int(abs(speed)), 100)
        self.pwm.pulse_widt

Example:

In [1]:
from tb6612 import TB6612
from pyb import Pin, Timer

# motor power control
nstby = Pin('NSTBY', mode=Pin.OUT_PP)
nstby.value(0)

# motors
pwm_timer = Timer(8, freq=10_0000)

motor1 = TB6612(
    pwm_timer.channel(3, Timer.PWM_INVERTED, pin=Pin('PWM_A')),
    Pin('AIN1', mode=Pin.OUT_PP),
    Pin('AIN2', mode=Pin.OUT_PP)
)

motor2 = TB6612(
    pwm_timer.channel(1, Timer.PWM_INVERTED, pin=Pin('PWM_B')),
    Pin('BIN1', mode=Pin.OUT_PP),
    Pin('BIN2', mode=Pin.OUT_PP)
)

Start motors: if everything is wired correctly, the motors will spin when running the code in next cell.

In [1]:
from time import sleep

# enable TB6612
nstby.value(1)

for speed in range(0, 101, 20):
    # set desired speed (PWM duty cycle)
    motor1.speed(speed)
    motor2.speed(-speed // 2)
    time.sleep(1)
    
nstby.value(0)

### Encoder

Just like a car, motor speed depends on load. Encoders can be used to measure the actual speed.


I've chosen magnetic encoders:

```{figure} figures/encoder.jpg
:width: 300px
:name: h-bridge

[Magnetic Encoder](https://www.pololu.com/product/3081) breakout boards.
```

A wheel with 12 magnets with alternating magnetization mounted to the shaft to the motor passes two magnetic sensors (the black components on the board) to produce electrical pulses. The STM32 has built-in counters to tally up these pulses, realizing in effect an odometer.

Attention: this odometer measures position, rather than distance traveled - going backwards decreases the output! Presumably not what a car odometer measures, although I have to admit I never tried driving my car backwards on the highway to check if the odometer count decreases ...

In [1]:
!cat code/stm32/lib/encoder.py

# https://github.com/iot49/upy-examples/blob/master/encoder2.py

"""
Hardware quadrature encoder.
Works for Timers 2-5
  2 & 5 have 32-bit counters
  3 & 4 have 16-bit counters
Input is on ch1 & ch2.

Sample usage:

enc1 = init_encoder(3, Pin.cpu.A6, Pin.cpu.A7, Pin.AF2_TIM3)
enc2 = init_encoder(4, Pin.cpu.B6, Pin.cpu.B7, Pin.AF2_TIM4)

Readout (c2, 2's complement, only for 16-bit timers):
c2(enc1.counter())

Reset:
enc1.counter(0)
"""

from pyb import Pin, Timer

def init_encoder(timer, ch1_pin, ch2_pin, af):
    # sets up encoder and returns timer
    # read count by calling timer.counter()
    pin_a = Pin(ch1_pin, Pin.AF_PP, pull=Pin.PULL_NONE, af=af)
    pin_b = Pin(ch2_pin, Pin.AF_PP, pull=Pin.PULL_NONE, af=af)
    enc_timer = Timer(timer, prescaler=0, period=0xffff)
    enc_channel = enc_timer.channel(1, Timer.ENC_AB)
    return enc_timer


def c2(x):
    # two's complement of 16-Bit number
    if x & 0x8000:
        # two's complement of 16 bit value
        x -= 0x10000
    ret

In [1]:
from encoder import init_encoder, c2

enc1 = init_encoder(4, 'ENC_A1', 'ENC_A2', Pin.AF2_TIM4)
enc2 = init_encoder(3, 'ENC_B1', 'ENC_B2', Pin.AF2_TIM3)

A complete example:

In [1]:
# enable TB6612
nstby.value(1)

for speed in range(0, 101, 20):
    # set desired speed (PWM duty cycle)
    motor1.speed(speed)
    motor2.speed(-speed // 2)
    for i in range(5):
        print(f"{c2(enc1.counter()):6d}  {c2(enc2.counter()):6d}")
        time.sleep(0.1)
    
nstby.value(0)

     0       0
     0       0
     0       0
     0       0
     0       0
     0       0
   -44      -3
  -104      -3
  -170      -3
  -238      -3
  -307      -3
  -460     -54
  -643    -125
  -828    -199
 -1014    -274
 -1195    -351
 -1451    -468
 -1740    -600
 -2040    -734
 -2344    -868
 -2649   -1003
 -3032   -1178
 -3442   -1368
 -3855   -1557
 -4272   -1748
 -4689   -1941
 -5183   -2175
 -5715   -2426
 -6257   -2679
 -6792   -2932


## IMU

![](figures/BNO055_orientation.png)

![](figures/yaw_pitch_roll.png)

## PID

![](figures/PID_en.svg)
